# Generate CT slice video
Different windows with optional segmentations

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed
from pathlib import Path
from tqdm import tqdm

import nibabel as nib
import SimpleITK as sitk
import numpy as np
from skimage import measure

import matplotlib.pyplot as plt

from plot_utils import plot_contours, plot_text, plot_slice_full, get_slice_range, plot_and_save

In [ ]:
# Load CT
ct_folder = Path("/home/liushifeng/Desktop/AutoPET Lymphoma/dataset B_lymphoma/PETCT_1fc35d02da/08-09-2003-NA-PET-CT Ganzkoerper  primaer mit KM-36800/")
ct_folder = Path("/home/liushifeng/Desktop/AutoPET Lymphoma/dataset B_lymphoma/PETCT_0f44cec2e6/09-07-2003-NA-PET-CT Ganzkoerper  primaer mit KM-99224/")

ct_file = "CT.nii.gz"
ct_img = sitk.ReadImage(ct_folder / ct_file)

# Load segmentations
seg_folder = Path("/home/liushifeng/Desktop/AutoPET Lymphoma/SEG_Lymphoma_B_Rifki-selected/")
seg_file = "SEG_1fc35d02da.nii.gz.nii.seg.nrrd"
seg_file = "SEG_0f44cec2e6.nii.gz.nii.seg.nrrd"

seg_img = sitk.ReadImage(seg_folder / seg_file)
seg_img = sitk.Resample(seg_img, ct_img, sitk.Transform(), sitk.sitkNearestNeighbor, 0.0, seg_img.GetPixelID())
seg_data = sitk.GetArrayFromImage(seg_img)

# Get arrays
ct_array = sitk.GetArrayFromImage(ct_img)
seg_array = sitk.GetArrayFromImage(seg_img)

In [ ]:
slice_range, n_slices = get_slice_range(seg_array)

In [ ]:
image_id = str(Path("_".join(ct_folder.parts[-3:-1])))
frames_folder = Path("../data") / image_id
frames_folder.mkdir(exist_ok=True)
print(frames_folder)

In [ ]:
# NOTE: cannot be stopped after starting!
with ThreadPoolExecutor(max_workers=16) as e:
    for i in tqdm(range(*slice_range)):
        args = ct_array, seg_array, i, slice_range, frames_folder
        e.submit(plot_and_save, args)

In [ ]:
# visualize a slice
i = 1360
fig = plot_slice_full(ct_array, seg_array, i, slice_range)
fig.savefig(f"test2.jpg", bbox_inches='tight', pad_inches=0, dpi=180);

In [ ]:
ct_folder

In [ ]:
from video_utils import create_mp4

In [ ]:
import arrow

output_folder = Path("../outputs/")
date_str = arrow.now().format("YYYY-MM-DD")
output_path = output_folder / f"video{date_str}_{image_id}.mp4"

create_mp4(output_path, frames_folder)
print(f"Video saved as {output_path}")

## Window/visualize

In [ ]:
# Define window settings
lung_window = {'center': -600, 'width': 1500}
abdomen_window = {'center': 40, 'width': 400}

def apply_window(image, center, width):
    window_min = center - width // 2
    window_max = center + width // 2
    return np.clip(image, window_min, window_max)

In [ ]:
# Apply windowing
lung_ct = apply_window(ct_array, lung_window['center'], lung_window['width'])
abdomen_ct = apply_window(ct_array, abdomen_window['center'], abdomen_window['width'])

# Normalize for display
lung_ct = (lung_ct - lung_ct.min()) / (lung_ct.max() - lung_ct.min())
abdomen_ct = (abdomen_ct - abdomen_ct.min()) / (abdomen_ct.max() - abdomen_ct.min())

In [ ]:
# Select a slice for visualization (e.g., middle slice)
slice_idx = 1360

# Create overlay
# fig, ax = plt.subplots()
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

ax1.imshow(lung_ct[slice_idx], cmap='gray')
# ax1.imshow(seg_data[slice_idx], alpha=0.3, cmap='Reds')
ax1.set_title('Lung Window with Segmentation Overlay')

ax2.imshow(abdomen_ct[slice_idx], cmap='gray')
# ax2.imshow(seg_data[slice_idx], alpha=0.3, cmap='Reds')
ax2.set_title('Abdomen Window with Segmentation Overlay')

plt.show()